# Computational Graphs &mdash; A Demo Application

In [15]:
import torch

In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('Current device:', device)
if device.type == 'cuda':
    print('Current device name:', torch.cuda.get_device_name())
    print('CUDA device count:', torch.cuda.device_count())

Current device: cpu


In [17]:
my_tensor = torch.tensor(1., requires_grad=True)
print(my_tensor)


tensor(1., requires_grad=True)
